In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [11]:
from sklearn.model_selection import GridSearchCV, cross_val_predict

# ------------------ Decision Tree Classifier ------------------
param_grid = {
    'criterion': ['gini', 'entropy'],   # split quality measure
    'max_depth': [None, 5, 10, 20],     # tree depth
    'min_samples_split': [2, 5, 10]     # minimum samples for split
}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X,y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 20],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1_weighted', verbose=3)

In [12]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [13]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [14]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 10}
The f1_macro: 0.8657115702536987
The confusion Matrix:
 [[238  19]
 [ 34 109]]
The report:
               precision    recall  f1-score   support

           0       0.88      0.93      0.90       257
           1       0.85      0.76      0.80       143

    accuracy                           0.87       400
   macro avg       0.86      0.84      0.85       400
weighted avg       0.87      0.87      0.87       400



In [15]:
# Save Best Model
import pickle
filename = "decission_tree_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [16]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002993,0.000524,0.011784,0.001380,gini,None,2,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.716667,0.885893,0.799685,0.778516,0.791490,0.794450,0.054226,22
1,0.003118,0.000818,0.008627,0.000289,gini,None,5,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.716667,0.858913,0.857479,0.791113,0.803125,0.805459,0.052266,17
2,0.003179,0.001236,0.005747,0.001833,gini,None,10,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.783072,0.922545,0.855447,0.803611,0.681234,0.809182,0.080094,13
3,0.002351,0.000062,0.008336,0.006015,gini,5,2,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.783072,0.922545,0.869545,0.791113,0.753771,0.824010,0.062409,8
4,0.002971,0.001165,0.006472,0.002476,gini,5,5,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.716667,0.922545,0.869545,0.791113,0.753771,0.810728,0.075417,12
